Submit here:
https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw03

## Question 1 (1 point): Dummies on Month and Week-of-Month

Find the ABSOLUTE CORRELATION VALUE of the most correlated dummy <month-week_of_month> with the binary outcome variable is_positive_growth_5d_future?

You saw in the correlation analysis and modeling that September and October may be important seasonal months. In this task, we'll go futher and try to generate dummies for Month and Week-of-month (starting from 1). For example, the first week of October should be coded similar to this: 'October_w1'. Once you've generated the new set of variables, find the most correlated (in absolute value) one with is_positive_growth_5d_future and round it to 3 digits after the comma.

NOTE: new dummies will be used as features in the next tasks, please leave them in the dataset.

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt
     

In [2]:
df_full = pd.read_parquet("stocks_df_combined_2024_05_07.parquet.brotli", )

In [3]:
df = df_full[df_full.Date>='2000-01-01']


In [4]:
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']

CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type']

TO_PREDICT = [g for g in df_full.keys() if (g.find('future')>=0)]

TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV

In [5]:
df['ln_volume'] = df.Volume.apply(lambda x: np.log(x))

/tmp/ipykernel_8166/1818720232.py:1: RuntimeWarning: divide by zero encountered in log
  df['ln_volume'] = df.Volume.apply(lambda x: np.log(x))
/tmp/ipykernel_8166/1818720232.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ln_volume'] = df.Volume.apply(lambda x: np.log(x))


In [6]:
 # manually defined features
GROWTH = [g for g in df_full.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
GROWTH

CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']
     

TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
 'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']

TECHNICAL_PATTERNS = [g for g in df_full.keys() if g.find('cdl')>=0]
print(f'Technical patterns count = {len(TECHNICAL_PATTERNS)}, examples = {TECHNICAL_PATTERNS[0:5]}')

MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
 'DGS1', 'DGS5', 'DGS10']
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO

Technical patterns count = 61, examples = ['cdl2crows', 'cdl3blackrows', 'cdl3inside', 'cdl3linestrike', 'cdl3outside']


In [7]:
df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

/tmp/ipykernel_8166/2159284025.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['January' 'January' 'January' ... 'May' 'May' 'May']' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.loc[:,'Month'] = df.Month.dt.strftime('%B')
/tmp/ipykernel_8166/2159284025.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '1' '2' ... '4' '0' '1']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[:,'Weekday'] = df.Weekday.astype(str)


In [8]:
df['month_wom'] = df.apply(lambda row: f'{row["Month"]}-{(row["Date"].day-1)//7+1}', axis=1)

/tmp/ipykernel_8166/1038171603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month_wom'] = df.apply(lambda row: f'{row["Month"]}-{(row["Date"].day-1)//7+1}', axis=1)


In [9]:
df['month_wom']

3490    January-1
3491    January-1
3492    January-1
3493    January-1
3494    January-1
          ...    
5422      April-5
5423        May-1
5424        May-1
5425        May-1
5426        May-1
Name: month_wom, Length: 182675, dtype: object

In [10]:
CATEGORICAL.append('month_wom')

In [11]:
dummy_variables = pd.get_dummies(df[CATEGORICAL], dtype='int32')


In [12]:
print(f'We have {len(dummy_variables.columns)} dummy vars')

We have 115 dummy vars


In [13]:
DUMMIES = dummy_variables.columns

In [14]:
df_with_dummies = pd.concat([df, dummy_variables], axis=1)

In [15]:
df_with_corr = df_with_dummies[DUMMIES].corrwith(df['is_positive_growth_5d_future'])

In [16]:
df_with_corr.apply(np.abs).sort_values(ascending=False)

month_wom_September-3    0.034537
month_wom_March-4        0.026058
month_wom_October-5      0.026023
month_wom_February-3     0.024578
month_wom_May-4          0.022264
                           ...   
Ticker_RELIANCE.NS       0.000106
Ticker_TTE               0.000063
month_wom_February-1     0.000059
month_wom_March-2        0.000036
month_wom_November-1     0.000033
Length: 115, dtype: float64

In [17]:
print(f'The highest value is {df_with_corr.apply(np.abs).sort_values(ascending=False).iloc[0]}')

The highest value is 0.03453687650579432


## Q2: 
* You're asked to define two new 'hand' rules (leading to 'positive' subtrees):

* pred3_manual_gdp_fastd: (gdppot_us_yoy <= 0.027) & (fastd >= 0.251)
* pred4_manual_gdp_wti_oil: (gdppot_us_yoy >= 0.027) & (growth_wti_oil_30d <= 1.005)
* Extend the Code Snippet 3 (Manual "hand rule" predictions): Calculate and add new rules (pred3 and pred4) to the dataframe.You should notice that one of the predictions doesn't have any positive predictions on TEST dataset (while it has many on TRAIN+VALIDATION).

* Debug: check in the new_df and the original dataset/data generation process that we didn't make any mistakes during the data transformation step.

* Explain why this can happen even if there are no errors in the data features.

* As a result, write down the precision score for the remaining predictor (round to three decimal points). E.g. if you have 0.57897, your answer should be 0.579.

In [18]:
def temporal_split(df, min_date, max_date, train_prop=0.7, val_prop=0.15, test_prop=0.15):
    """
    Splits a DataFrame into three buckets based on the temporal order of the 'Date' column.

    Args:
        df (DataFrame): The DataFrame to split.
        min_date (str or Timestamp): Minimum date in the DataFrame.
        max_date (str or Timestamp): Maximum date in the DataFrame.
        train_prop (float): Proportion of data for training set (default: 0.6).
        val_prop (float): Proportion of data for validation set (default: 0.2).
        test_prop (float): Proportion of data for test set (default: 0.2).

    Returns:
        DataFrame: The input DataFrame with a new column 'split' indicating the split for each row.
    """
    # Define the date intervals
    train_end = min_date + pd.Timedelta(days=(max_date - min_date).days * train_prop)
    val_end = train_end + pd.Timedelta(days=(max_date - min_date).days * val_prop)

    # Assign split labels based on date ranges
    split_labels = []
    for date in df['Date']:
        if date <= train_end:
            split_labels.append('train')
        elif date <= val_end:
            split_labels.append('validation')
        else:
            split_labels.append('test')

    # Add 'split' column to the DataFrame
    df['split'] = split_labels

    return df

min_date_df = df_with_dummies.Date.min()
max_date_df = df_with_dummies.Date.max()

df_with_dummies = temporal_split(df_with_dummies,
                                 min_date = min_date_df,
                                 max_date = max_date_df)

In [19]:
# generate manual predictions
# Let's label all prediction features with prefix "pred"
new_df = df_with_dummies.copy()
new_df['pred0_manual_cci'] = (new_df.cci>200).astype(int)
new_df['pred1_manual_prev_g1'] = (new_df.growth_1d>1).astype(int)
new_df['pred2_manual_prev_g1_and_snp'] = ((new_df['growth_1d'] > 1) & (new_df['growth_snp500_1d'] > 1)).astype(int)

In [20]:
new_df['pred3_manual_gdp_fastd'] = ((new_df['gdppot_us_yoy'] <= 0.027) & (new_df['fastd'] >= 0.251)).astype(int)

In [21]:
new_df['pred4_manual_gdp_wti_oil']=((new_df['gdppot_us_yoy']>=0.027) & (new_df['growth_wti_oil_30d'] <= 1.005)).astype(int)

In [22]:
rule_thumb_prediction = new_df[new_df["split"] == 'test'][['pred4_manual_gdp_wti_oil', 'pred3_manual_gdp_fastd', 'is_positive_growth_5d_future']]

In [23]:
rule_thumb_prediction[['pred4_manual_gdp_wti_oil', 'pred3_manual_gdp_fastd']].nunique()

pred4_manual_gdp_wti_oil    1
pred3_manual_gdp_fastd      2
dtype: int64

In [24]:
rule_thumb_prediction['pred4_manual_gdp_wti_oil'].unique(),  rule_thumb_prediction['pred3_manual_gdp_fastd'].unique()

(array([0]), array([1, 0]))

In [25]:
print(f'Accuracy {round((rule_thumb_prediction["pred3_manual_gdp_fastd"]==rule_thumb_prediction["is_positive_growth_5d_future"]).sum()/len(rule_thumb_prediction),3)}')

Accuracy 0.555


## Question 3 (1 point): Unique correct predictions from a 10-levels deep Decision Tree Classifier (pred5_clf_10)

* What is the total number of records in the TEST dataset when the new prediction pred5_clf_10 is better than all 'hand' rules (pred0..pred4)?

NOTE: please include random_state=42 to Decision Tree Classifier init function (line clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)) to ensure everyone gets the same results.

Suggested solution:

Step1: Rewrite the '1.4.3 Inference for a decision tree' piece for the Decision Tree Classifier with max_depth=10 (clf_10), so that you fit the model on TRAIN+VALIDATION sets (unchanged from the lecture), but predict on the whole set X_all (to be able to define a new column 'pred5_clf_10' in the dataframe new_df). Here is the link with explanation. It will solve the problem in 1.4.5 when predictions were made only for Test dataset and couldn't be easily joined with the full dataset.

Step2: Once you have it, define a new column 'only_pred5_is_correct' similar to 'hand' prediction rules with several conditions: is_positive_growth_5d_future AND is_correct_pred5 should be equal 1, while all other predictions is_correct_pred0..is_correct_pred4 should be equal to 0.

Step3: Convert 'only_pred5_is_correct' column from bool to int, and find how many times it is equal to 1 in the TEST set. Write down this as an answer.

ADVANCED: define a function that can be applied to the whole row of predictions (a few examples of pandas-apply-row-functions) and can find whether some prediction 'predX' (where X is one of the predictions) is uniquely correct. It should work even if there are 100 predictions available, so that you don't define manually the condition for 'predX'.

In [26]:
import numpy as np

def remove_infinite_values(X):
    """
    Remove infinite values from the input array.

    Parameters:
    - X: Input array (NumPy array or array-like)

    Returns:
    - Array with infinite values removed
    """
    return X[np.isfinite(X).all(axis=1)]

# Example usage:
# Assuming X is your input data
# filtered_X = remove_infinite_values(X)

In [27]:
DUMMIES.to_list()

['Month_April',
 'Month_August',
 'Month_December',
 'Month_February',
 'Month_January',
 'Month_July',
 'Month_June',
 'Month_March',
 'Month_May',
 'Month_November',
 'Month_October',
 'Month_September',
 'Weekday_0',
 'Weekday_1',
 'Weekday_2',
 'Weekday_3',
 'Weekday_4',
 'Weekday_5',
 'Weekday_6',
 'Ticker_AAPL',
 'Ticker_ACN',
 'Ticker_AMZN',
 'Ticker_ASML',
 'Ticker_AVGO',
 'Ticker_BHARTIARTL.NS',
 'Ticker_BRK-B',
 'Ticker_CDI.PA',
 'Ticker_GOOG',
 'Ticker_HDB',
 'Ticker_HINDUNILVR.NS',
 'Ticker_IBN',
 'Ticker_IDEXY',
 'Ticker_INFY',
 'Ticker_ITC.NS',
 'Ticker_JPM',
 'Ticker_LICI.NS',
 'Ticker_LLY',
 'Ticker_LT.NS',
 'Ticker_MC.PA',
 'Ticker_META',
 'Ticker_MSFT',
 'Ticker_NVDA',
 'Ticker_NVO',
 'Ticker_OR.PA',
 'Ticker_RELIANCE.NS',
 'Ticker_RMS.PA',
 'Ticker_SAP',
 'Ticker_SBIN.NS',
 'Ticker_SIE.DE',
 'Ticker_TCS.NS',
 'Ticker_TTE',
 'Ticker_V',
 'ticker_type_EU',
 'ticker_type_INDIA',
 'ticker_type_US',
 'month_wom_April-1',
 'month_wom_April-2',
 'month_wom_April-3',
 'month

In [28]:
# Split the data into training and testing sets based on the split date
features_list = NUMERICAL+DUMMIES.to_list()
to_predict = 'is_positive_growth_5d_future'

train_df = new_df[new_df.split.isin(['train','validation'])].copy(deep=True)
test_df = new_df[new_df.split.isin(['test'])].copy(deep=True)

# ONLY numerical Separate features and target variable for training and testing sets
# need Date and Ticker later when merging predictions to the dataset
X_train = train_df[features_list+[to_predict,'Date','Ticker']]
X_test = test_df[features_list+[to_predict,'Date','Ticker']]

print(f'length: X_train {X_train.shape},  X_test {X_test.shape}')

length: X_train (152846, 302),  X_test (29829, 302)


In [29]:
# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Need to fill NaNs somehow
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

print(f'length: X_train_imputed {X_train.shape},  X_test_imputed {X_test.shape}')

length: X_train_imputed (152846, 302),  X_test_imputed (29829, 302)


In [33]:
y_train = X_train[to_predict]
y_test = X_test[to_predict]

# remove y_train, y_test from X_ dataframes
del X_train[to_predict]
del X_test[to_predict]

In [56]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score
def fit_decision_tree(X, y, max_depth=20):
# Initialize the Decision Tree Classifier
  clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)

  # Fit the classifier to the training data
  clf.fit(X, y)
  return clf, X.columns

In [34]:
clf_10, train_columns = fit_decision_tree(X=X_train.drop(['Date','Ticker'],axis=1),
                           y=y_train,
                           max_depth=10)

In [37]:
new_df['pred5_clf10'] = clf_10.predict(new_df[features_list].replace([np.inf, -np.inf], 0))

In [43]:
pred_columns = [_pred  for _pred in new_df.columns if 'pred' in _pred]
pred_columns.remove( 'pred5_clf10')
pred_columns

['pred0_manual_cci',
 'pred1_manual_prev_g1',
 'pred2_manual_prev_g1_and_snp',
 'pred3_manual_gdp_fastd',
 'pred4_manual_gdp_wti_oil']

In [51]:
def only_tree_is_correct(row):
    if (row['pred5_clf10']!=1) and (row['is_positive_growth_5d_future']!=1):
        return False
    for _pred_col in pred_columns:
        if row[_pred_col]==1:
            return False
    
    return True    

new_df[new_df.split == 'test'].apply(only_tree_is_correct, axis=1).sum()

5

Question 4: (2 points) Hyperparameter tuning for a Decision Tree
What is the optimal tree depth (from 1 to 20) for a DecisionTreeClassifier?

NOTE: please include random_state=42 to Decision Tree Classifier init function (line clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)) to ensure consistency in results.

Follow these steps to find the optimal max_depth:

Iterate through max_depth values from 1 to 20.
Train the Decision Tree Classifier with the current max_depth parameter.
Optionally, visualize how the 'head' of each fitted tree changes with more advanced (=deep) trees. You can use the sklearn.tree.plot_tree() function, or the compact way with the export_text() functionality (Stack Overflow example):
from sklearn.tree import export_text
tree_rules = export_text(model, feature_names=list(X_train), max_depth=3)
print(tree_rules)
Calculate the precision score (you can use the function sklearn.metrics.precision_score()) on the TEST dataset for each of the fitted trees. You can also compare it with the precision score on a VALIDATION dataset, which is included to the training phase (to have more data to train on). You should see that the precision score on a VALIDATION set starts to grow with the complexity of a tree (overfit), which isn't seen on the precision score on TEST.
Identify the optimal max_depth, where the precision score is the highest on the TEST dataset. Record this value as best_max_depth and submit as an answer.
Make predictions on all records (TRAIN+VALIDATION+TEST) and add the new prediction pred6_clf_best to the dataframe new_df.
Additionally, compare the precision score of the tuned decision tree with previous predictions. You should observe an improvement (>0.58, or more than 58% precision), indicating that the tuned tree outperforms previous manual "hand" rules and Decision Tree predictions.

In [55]:
from collections import defaultdict
trees_result = defaultdict(int)
for k in range(1, 21):
    print(f'Training tree with depth {k}')
    clf, train_columns = fit_decision_tree(X=X_train.drop(['Date','Ticker'],axis=1),
                           y=y_train,
                           max_depth=k)
    print(f'Accuracy on test: {precision_score(y_test, clf.predict(X_test.drop(columns=["Date", "Ticker"])))}')
    
    trees_result[k] = precision_score(y_test, clf.predict(X_test.drop(columns=["Date", "Ticker"])))
    
    y_val = new_df[new_df.split == 'validation'][to_predict]
    X_val = new_df[new_df.split == 'validation'][features_list].replace([np.inf, -np.inf], 0)
    print(f'Accuracy on validation: {precision_score(y_val, clf.predict(X_val))}')

    

Training tree with depth 1


Accuracy on test: 0.5551979617151095
Accuracy on validation: 0.5717299578059072
Training tree with depth 2
Accuracy on test: 0.5553655838278185
Accuracy on validation: 0.5717299578059072
Training tree with depth 3
Accuracy on test: 0.5551979617151095
Accuracy on validation: 0.5696883081529877
Training tree with depth 4
Accuracy on test: 0.5552650105601931
Accuracy on validation: 0.5696883081529877
Training tree with depth 5
Accuracy on test: 0.5556002547856114
Accuracy on validation: 0.5696883081529877
Training tree with depth 6
Accuracy on test: 0.5677696201682926
Accuracy on validation: 0.567442493534776
Training tree with depth 7
Accuracy on test: 0.5650206175198632
Accuracy on validation: 0.5721382877364911
Training tree with depth 8
Accuracy on test: 0.5650206175198632
Accuracy on validation: 0.5760514495712535
Training tree with depth 9
Accuracy on test: 0.5669985584498307
Accuracy on validation: 0.5781951817068192
Training tree with depth 10
Accuracy on test: 0.5570418049549096


KeyboardInterrupt: 